In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE57691"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE57691"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE57691.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE57691.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE57691.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differential gene expression in human abdominal aortic aneurysm and atherosclerosis"
!Series_summary	"The aim of this study was to assess the relative gene expression in human AAA and AOD."
!Series_overall_design	"Genome-wide expression analysis of  abdominal aortic aneurysm (AAA) and aortic occlusive disease (AOD) specimens obtained from 20 patients with small AAA (mean maximum aortic diameter=54.3±2.3 mm), 29 patients with large AAA (mean maximum aortic diameter=68.4±14.3 mm), and 9 AOD patients (mean maximum aortic diameter=19.6±2.6 mm). Relative aortic gene expression was compared with that of 10 control aortic specimen of organ donors."
Sample Characteristics Dictionary:
{0: ['disease state: small AAA', 'disease state: large AAA', 'disease state: AOD', 'disease state: control'], 1: ['subjects: patients with AAA undergoing open surgery to treat AAA', 'subjects: patients with AOD undergoing open surgery to treat chronic lower limb ischemia', 's

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# Study title mentions "Differential gene expression"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, we can see:
# - disease state (related to Atherosclerosis) is at index 0
# - Age is not available 
# - Gender is not available
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary format (0 for control, 1 for disease)"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert values
    if 'control' in value.lower():
        return 0
    elif 'aod' in value.lower() or 'aaa' in value.lower():
        # Both AOD (Aortic Occlusive Disease) and AAA (Abdominal Aortic Aneurysm)
        # represent cases of atherosclerosis
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous format"""
    # Since age data is not available, this function is a placeholder
    return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)"""
    # Since gender data is not available, this function is a placeholder
    return None

# 3. Save Metadata
# Determine if trait data is available (based on whether trait_row is None)
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Assuming clinical_data has been loaded in a previous step
    try:
        # Ensure directory exists
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Read the clinical data from the input directory
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the selected clinical data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If clinical data file doesn't exist, log the error
        if not os.path.exists(os.path.join(in_cohort_dir, "clinical_data.csv")):
            print("Clinical data file not found. This may be expected if data was extracted differently.")


Error processing clinical data: [Errno 2] No such file or directory: '../../input/GEO/Atherosclerosis/GSE57691/clinical_data.csv'
Clinical data file not found. This may be expected if data was extracted differently.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE57691/GSE57691_series_matrix.txt.gz


Gene data shape: (39426, 68)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers seen in the gene data start with "ILMN_" which indicates these are Illumina microarray probe IDs
# These are not human gene symbols and need to be mapped to gene symbols for proper analysis
# ILMN_ identifiers are specific to Illumina BeadArray technology and require mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's explore the SOFT file more thoroughly to find gene symbols
print("\nExploring SOFT file more thoroughly for gene information:")
gene_info_patterns = []
entrez_to_symbol = {}

with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000:  # Check header section for platform info
            if '!Series_platform_id' in line or '!Platform_title' in line:
                print(line.strip())
                
        # Look for gene-related columns and patterns in the file
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line or 'Symbol' in line:
            gene_info_patterns.append(line.strip())
            
        # Extract a mapping using ENTREZ_GENE_ID if available
        if len(gene_info_patterns) < 2 and 'ENTREZ_GENE_ID' in line and '\t' in line:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                try:
                    # Attempt to add to mapping - assuming ENTREZ_GENE_ID could help with lookup
                    entrez_id = parts[1]
                    probe_id = parts[0]
                    if entrez_id.isdigit() and entrez_id != probe_id:
                        entrez_to_symbol[probe_id] = entrez_id
                except:
                    pass
        
        if i > 10000 and len(gene_info_patterns) > 0:  # Limit search but ensure we found something
            break

# Show some of the patterns found
if gene_info_patterns:
    print("\nFound gene-related patterns:")
    for pattern in gene_info_patterns[:5]:
        print(pattern)
else:
    print("\nNo explicit gene info patterns found")

# Let's try to match the ENTREZ_GENE_ID to the probe IDs
print("\nAnalyzing ENTREZ_GENE_ID column:")
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    # Check if ENTREZ_GENE_ID contains actual Entrez IDs (different from probe IDs)
    gene_annotation['ENTREZ_GENE_ID'] = gene_annotation['ENTREZ_GENE_ID'].astype(str)
    different_ids = (gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']).sum()
    print(f"Number of entries where ENTREZ_GENE_ID differs from ID: {different_ids}")
    
    if different_ids > 0:
        print("Some ENTREZ_GENE_ID values differ from probe IDs - this could be useful for mapping")
        # Show examples of differing values
        diff_examples = gene_annotation[gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']].head(5)
        print(diff_examples)
    else:
        print("ENTREZ_GENE_ID appears to be identical to probe ID - not useful for mapping")

# Search for additional annotation information in the dataset
print("\nLooking for alternative annotation approaches:")
print("- Checking for platform ID or accession number in SOFT file")

platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Platform_geo_accession' in line:
            platform_id = line.split('=')[1].strip().strip('"')
            print(f"Found platform GEO accession: {platform_id}")
            break
        if i > 200:
            break

# If we don't find proper gene symbol mappings, prepare to use the ENTREZ_GENE_ID as is
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    print("\nPreparing provisional gene mapping using ENTREZ_GENE_ID:")
    mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
    mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'}, inplace=True)
    print(f"Provisional mapping data shape: {mapping_data.shape}")
    print(preview_df(mapping_data, n=5))
else:
    print("\nWarning: No suitable mapping column found for gene symbols")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene expression data and gene annotation data to identify matching columns
print("\nDetermining which columns to use for gene mapping:")

# Based on the gene expression data, we're using 'ID' as the identifier (ILMN_* format)
# From the annotation data preview, 'Symbol' contains gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

print(f"Using '{prob_col}' column as probe identifier and '{gene_col}' column as gene symbol")

# 2. Extract the gene mapping dataframe using the identified columns
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping data shape: {mapping_data.shape}")
print("Mapping data preview:")
print(preview_df(mapping_data, n=5))

# Check if any mapping exists (non-empty mapping dataframe)
if mapping_data.empty:
    print("Warning: Empty mapping data. No valid probe-to-gene mappings found.")
else:
    print(f"Found {mapping_data.shape[0]} probe-to-gene mappings")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Save the gene data for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")



Determining which columns to use for gene mapping:
Using 'ID' column as probe identifier and 'Symbol' column as gene symbol
Gene mapping data shape: (44837, 2)
Mapping data preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Found 44837 probe-to-gene mappings


Gene expression data shape after mapping: (19295, 68)
First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1'],
      dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE57691.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Extract and prepare clinical data from the matrix file
print("\nPreparing clinical data...")

# Get the clinical data rows
_, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
_, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data using the parameters defined in Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=0,  # From Step 2: trait_row = 0
    convert_trait=convert_trait,  # Function defined in Step 2
    age_row=None,  # From Step 2: age_row = None
    convert_age=None,
    gender_row=None,  # From Step 2: gender_row = None
    convert_gender=None
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE57691 dataset contains gene expression data from patients with abdominal aortic aneurysm (AAA) and aortic occlusive disease (AOD) compared to control subjects. The dataset focuses on atherosclerosis-related vascular changes."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (18540, 68)
First 10 normalized gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE57691.csv

Preparing clinical data...
Clinical data preview:
{'GSM1386783': [1.0], 'GSM1386784': [1.0], 'GSM1386785': [1.0], 'GSM1386786': [1.0], 'GSM1386787': [1.0], 'GSM1386788': [1.0], 'GSM1386789': [1.0], 'GSM1386790': [1.0], 'GSM1386791': [1.0], 'GSM1386792': [1.0], 'GSM1386793': [1.0], 'GSM1386794': [1.0], 'GSM1386795': [1.0], 'GSM1386796': [1.0], 'GSM1386797': [1.0], 'GSM1386798': [1.0], 'GSM1386799': [1.0], 'GSM1386800': [1.0], 'GSM1386801': [1.0], 'GSM1386802': [1.0], 'GSM1386803': [1.0], 'GSM1386804': [1.0], 'GSM1386805': [1.0], 'GSM1386806': [1.0], 'GSM1386807': [1.0], 'GSM1386808': [1.0], 'GSM1386809': [1.0], 'GSM1386810': [1.0], 'GSM1386811': [1.0], 'GSM1386812': [1.0], 'GSM1386813': [1.0], 'GSM1386814': [1.0], 'GSM1386815': [1.0], 'GSM1386816': [1.0], 'GSM1386817': [1.0], 'GSM1386818': [1.0], 'GSM1386819': [1.0], 'GSM1386820': [1.0], 'GSM1386821': [1.0], 'GSM1386822': [1.0], 'GSM1386823': 

Linked data shape after handling missing values: (68, 18541)

Checking for bias in dataset features...
For the feature 'Atherosclerosis', the least common label is '0.0' with 10 occurrences. This represents 14.71% of the dataset.
The distribution of the feature 'Atherosclerosis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Atherosclerosis/GSE57691.csv
